### Initialize library

In [1]:
%load_ext autoreload
%autoreload 2

from olp_library import *
# import FOON_graph_analyser as fga

### Language to  OLP

In [2]:
models = [
    "gpt-3.5-turbo",
    "gpt-4",
    ]

in_context_file = "incontext_examples.txt"
user_task = "How do I make coffee?"
# user_task = "How do I make a burger?"
# user_task = "How do I unlock a door?"
# user_task = "How do I turn on the lights in my house"

high_level_plan, object_level_plan, plan_objects = generate_olp(user_task, in_context_file, models[1], verbose=True)

print(plan_objects)

*************************************************************************
Stage1 Prompting
*************************************************************************
Model: gpt-4
Complete prompt:
[
    {
        "role": "system",
        "content": "You are an LLM that understands how to generate concise high level plans for arbitrary tasks involving objects. Only focus on what happens to objects.  Stay consistent with object names and use one verb per step. Assume you have all objects necessary for the plan and do not need to obtain anything."
    },
    {
        "role": "user",
        "content": "How do I make coffee?\nAfter listing the high level plan, list all the unique objects used in the high level plan, including the object created after solving the task, ignoring state changes to those objects. Follow the format 'unique_objects:object_1, object_2, object_3, ...'"
    }
]
*************************************************************************
Stage 1 Response: 
1. Fill kettl

In [3]:
## Visualize Outputs
print("***********************\n   High level plan\n***********************\n",   high_level_plan,   "\n")
print("***********************\n   Object level plan\n***********************")
print(*object_level_plan, sep="\n")
print("\n***********************\n   Plan objects\n***********************\n",      plan_objects,      "\n")

***********************
   High level plan
***********************
 1. Fill kettle with water.
2. Heat kettle.
3. Place coffee grounds in coffee maker.
4. Pour heated water into coffee maker.
5. Start coffee maker.
6. Pour brewed coffee into cup.

unique_objects: kettle, water, coffee grounds, coffee maker, cup, brewed coffee. 

***********************
   Object level plan
***********************
{'Step': 'Fill kettle with water', 'Objects': ['kettle', 'water'], 'Action': 'Fill', 'StateChanges': {'kettle': {'Precondition': ['empty'], 'Effect': ['contains water']}, 'water': {'Precondition': ['free'], 'Effect': ['in kettle']}}}
{'Step': 'Heat kettle', 'Objects': ['kettle', 'water'], 'Action': 'Heat', 'StateChanges': {'kettle': {'Precondition': ['contains water'], 'Effect': ['contains water', 'hot']}, 'water': {'Precondition': ['in kettle'], 'Effect': ['in kettle', 'hot']}}}
{'Step': 'Place coffee grounds in coffee maker', 'Objects': ['coffee grounds', 'coffee maker'], 'Action': 'Place', 

In [4]:
## Test Foon creation
plan_step = object_level_plan[0]

sample_unit = create_olp_functionalUnit(plan_step,plan_objects)
sample_unit.print_functions[2](version=1)

Creating functional unit for: Fill kettle with water
Objects: ['kettle', 'water']
Current object is : kettle
	 kettle state changes: {'Precondition': ['empty'], 'Effect': ['contains water']}
		 Precondition : empty || related objects: []
		 Effect : contains water || related objects: None
Current object is : water
	 water state changes: {'Precondition': ['free'], 'Effect': ['in kettle']}
		 Precondition : free || related objects: []
		 Effect : in kettle || related objects: ['kettle']
O	kettle
S	empty
O	water
S	free
M	Fill	<Assumed>
O	kettle
S	contains 	{water}
O	water
S	in 	 [kettle]


In [5]:
#Complete FOON prototype:
FOON_prototype = []

for step in object_level_plan:
    print(step['Step'])
    # -- now we will create functional units that follow the FOON format:
    new_unit = create_olp_functionalUnit(step, plan_objects)      
    new_unit.print_functions[2]()
    
    # input()
    print()

    # -- add the functional unit to the FOON prototype:
    FOON_prototype.append(new_unit)

Fill kettle with water
Creating functional unit for: Fill kettle with water
Objects: ['kettle', 'water']
Current object is : kettle
	 kettle state changes: {'Precondition': ['empty'], 'Effect': ['contains water']}
		 Precondition : empty || related objects: []
		 Effect : contains water || related objects: None
Current object is : water
	 water state changes: {'Precondition': ['free'], 'Effect': ['in kettle']}
		 Precondition : free || related objects: []
		 Effect : in kettle || related objects: ['kettle']
O	kettle
S	empty
O	water
S	free
M	Fill	<Assumed>
O	kettle
S	contains 	{water}
O	water
S	in 	 [kettle]

Heat kettle
Creating functional unit for: Heat kettle
Objects: ['kettle', 'water']
Current object is : kettle
	 kettle state changes: {'Precondition': ['contains water'], 'Effect': ['contains water', 'hot']}
		 Precondition : contains water || related objects: None
		 Effect : contains water || related objects: None
		 Effect : hot || related objects: []
Current object is : water
	

In [10]:
# -- save the prototype FOON graph as a text file, which we will then run with a parser to correct numbering:
if not os.path.exists('preprocess/'):
	os.makedirs('preprocess/')

if not os.path.exists('postprocess/'):
	os.makedirs('postprocess/')

file_ = open('preprocess/prototype.txt', 'w')
file_.write('#\tFOON Prototype\n#\t-- Task Prompt: {0}\n//\n'.format(user_task))
for unit in FOON_prototype:
	file_.write(unit.getFunctionalUnitText())
	unit.print_functions[2]()
	print('//')

file_.close()

# -- running parsing module to ensure that FOON labels and IDs are made consistent for further use:
#		(it is important that each object and state type have a *UNIQUE* identifier)
fpa.skip_JSON_conversion = True		# -- we don't need JSON versions of a FOON
fpa.skip_index_check = True			# -- always create a new set of index files

fpa.source_dir = './preprocess/'	
fpa.target_dir = './postprocess/'
fpa._run_parser()

# -- after running the parser, take the 'prototype.txt' file and plot it using the following tool: https://davidpaulius.github.io/foon-view/
fga._startFOONview()

O	kettle
S	empty
O	water
S	free
M	Fill	<Assumed>
O	kettle
S	contains 	{water}
O	water
S	in 	 [kettle]
//
O	kettle
S	contains 	{water}
O	water
S	in 	 [kettle]
M	Heat	<Assumed>
O	kettle
S	contains 	{water}
S	hot
O	water
S	hot
S	in 	 [kettle]
//
O	coffee grounds
S	free
O	coffee maker
S	empty
M	Place	<Assumed>
O	coffee grounds
S	in 	 [coffee maker]
O	coffee maker
S	contains 	{coffee grounds}
//
O	water
S	hot
S	in 	 [kettle]
O	coffee maker
S	contains 	{coffee grounds}
O	kettle
S	contains 	{water}
S	hot
M	Pour	<Assumed>
O	water
S	in 	 [coffee maker]
O	coffee maker
S	contains 	{coffee grounds,water}
O	kettle
S	empty
//
O	coffee maker
S	contains 	{coffee grounds,water}
M	Start	<Assumed>
O	coffee maker
S	brewing
//
O	brewed coffee
S	in 	 [coffee maker]
O	cup
S	empty
O	coffee maker
S	brewing
M	Pour	<Assumed>
O	brewed coffee
S	in 	 [cup]
O	cup
S	contains 	{brewed coffee}
O	coffee maker
S	empty
//
-- [FOON_parser] : Initiating parsing procedure!


 -- [FOON_parser] : Commencing parsing...
  -- par